In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
# use large scale here to get the output really close to the value in dataset, which is important to Median.
# Be careful, since we may need to calculate average of 2 values in case of no middle value in dataset,
# this means larger scale can still blow up circuit size, unlike Mode func that scale doesnt affect circuit size much.
scales = [7]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.median(x)

# error here in Median only matters in determining the median value in case it doesnt exist in dataset. (Avg of 2 middle values)
error = 0.01
# Prover/ data owner side
_, prover_model = computation_to_model(computation,precal_witness_path, True, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Prover side
final op:  <zkstats.ops.Median object at 0x127707350>
==== Generate & Calibrate Setting ====


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:198: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


 <------------- Numerical Fidelity Report (input_scale: 7, param_scale: 7, scale_input_multiplier: 10) ------------->

+--------------+--------------+-------------+-----------+----------

scale:  [7]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[-24804,12494],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":15968,"total_const_size":4827,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"Floor":{"scale":2.0}},"ReLU",{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1714803491928}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

Verifier side
finish create
final op:  <zkstats.ops.Median object at 0x140564c90>


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 7.38695216178894 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.546875
==== Generating Proof ====
proof:  {'instances': [['13613be83876b1d6edaa4022f0037a4a0dd2bca4e9a736c5ced55f1d1ec6fb29', '0100000000000000000000000000000000000000000000000000000000000000', 'c618000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1e97941129f68464dd56eb9d579273bbe50981355862c052c406ecacd1bb5ad21645963caa1f2077ccfb530c8ad764b4305dd7a24749f015f19987a019349ba40d446cdd5cf7440e98f80804f80165d15fd3d318547dfb16ffe8c8c6a814b9a21f405b558c16b7f33dd5cf9df321ab92e0c4da36d024c564164550810a950d8d119ba7e5abae1ae75b83345bd70f275bcde52a5a2e2a999ad632653ba578154e2f7bba432aae48fc7bfac2e740024931213fef8283dbd5c0509208d5c91abfb015741f6b42a861b35c1a590cbbdfd4ce1c6ef33197aa73b929f1b04703f10e460ae4770b6dab02f37adbba20908e50aac7870393a731b71f7f61e2fb25ac621a049e9a60538b0b7123cd32525ec4fb63977b80c9c05fd764382bfb62b53d011c1b1e0a21b550

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [49.546875]
